Make sure to upload NER_TRAIN_PREAMBLE.json to colab runtime and have the model-best.zip and model-last.zip uploaded in your gdrive before running the below codes

model-best.zip link : https://drive.google.com/file/d/1JS78a_xz_maOeDN2kkAdbSc3NzWBJzlQ/view?usp=sharing

model-last.zip link: https://drive.google.com/file/d/1vhRJ4htbYn4rW7dSB3VskfpJvKxrxvYm/view?usp=sharing

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
!unzip gdrive/My\ Drive/preamble_models/model-best.zip

Archive:  gdrive/My Drive/preamble_models/model-best.zip
  inflating: model-best/meta.json    
  inflating: model-best/config.cfg   
  inflating: model-best/tokenizer    
  inflating: model-best/vocab/vectors.cfg  
  inflating: model-best/vocab/strings.json  
  inflating: model-best/vocab/lookups.bin  
  inflating: model-best/ner/model    
  inflating: model-best/vocab/vectors  
  inflating: model-best/vocab/key2row  
  inflating: model-best/ner/cfg      
  inflating: model-best/ner/moves    
  inflating: model-best/transformer/cfg  
  inflating: model-best/transformer/model  


In [ ]:
!unzip gdrive/My\ Drive/preamble_models/model-last.zip

Archive:  gdrive/My Drive/preamble_models/model-last.zip
  inflating: model-last/tokenizer    
  inflating: model-last/ner/moves    
  inflating: model-last/config.cfg   
  inflating: model-last/ner/model    
  inflating: model-last/meta.json    
  inflating: model-last/ner/cfg      
  inflating: model-last/transformer/cfg  
  inflating: model-last/vocab/key2row  
  inflating: model-last/vocab/vectors.cfg  
  inflating: model-last/vocab/strings.json  
  inflating: model-last/vocab/vectors  
  inflating: model-last/vocab/lookups.bin  
  inflating: model-last/transformer/model  


In [ ]:
pip install spacy-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 2.4 MB/s 
     |████████████████████████████████| 4.7 MB 49.1 MB/s 
     |████████████████████████████████| 1.1 MB 57.1 MB/s 
     |████████████████████████████████| 163 kB 72.8 MB/s 
     |████████████████████████████████| 6.6 MB 57.8 MB/s 


In [ ]:
import spacy

In [ ]:
from spacy import displacy

In [ ]:
import json

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_json(r'NER_TRAIN_PREAMBLE.json')
d=df.to_dict('records')
training_data=[]
for i in range(1000):
  temp_dict={}
  temp_dict['text'] = d[i]['data']['text']
  temp_dict['entities'] = []
  for annotation in d[i]['annotations'][0]['result']:
    start = annotation['value']['start']
    end = annotation['value']['end']
    label = annotation['value']['labels'][0].upper()
    temp_dict['entities'].append((start, end, label))
  training_data.append(temp_dict)
print(training_data[0])
testing_data=[]
for i in range(1000,1560):
  temp_dict={}
  temp_dict['text'] = d[i]['data']['text']
  temp_dict['entities'] = []
  for annotation in d[i]['annotations'][0]['result']:
    start = annotation['value']['start']
    end = annotation['value']['end']
    label = annotation['value']['labels'][0].upper()
    temp_dict['entities'].append((start, end, label))
  testing_data.append(temp_dict)

training_data_opt=[]
for i in range(0,len(training_data)):
  if (len(training_data[i]['text'])) < 1250:
    training_data_opt.append(training_data[i])

testing_data_opt=[]
for i in range(0,len(testing_data)):
  if (len(testing_data[i]['text'])) < 1250:
    testing_data_opt.append(training_data[i])



{'text': "In The High Court Of Kerala At Ernakulam\n\nCrl Mc No. 1622 of 2006()\n\n\n1. T.R.Ajayan, S/O. O.Raman,\n                      ...  Petitioner\n\n                        Vs\n\n\n\n1. M.Ravindran,\n                       ...       Respondent\n\n2. Mrs. Nirmala Dinesh, W/O. Dinesh,\n\n                For Petitioner  :Sri.A.Kumar\n\n                For Respondent  :Smt.M.K.Pushpalatha\n\nThe Hon'ble Mr. Justice P.R.Raman\nThe Hon'ble Mr. Justice V.K.Mohanan\n\n Dated :07/01/2008\n\n O R D E R\n", 'entities': [(7, 40, 'COURT'), (73, 83, 'PETITIONER'), (171, 182, 'RESPONDENT'), (237, 251, 'RESPONDENT'), (304, 311, 'LAWYER'), (350, 365, 'LAWYER'), (391, 400, 'JUDGE'), (425, 436, 'JUDGE')]}


In [ ]:
nlp_ner = spacy.load("/content/model-best/")

confusion matrix code

In [ ]:
confusion_matrix=pd.DataFrame(columns=['COURT','PETITIONER', 'RESPONDENT' , 'LAWYER', 'JUDGE' , 'NONE'])
confusion_matrix.loc['COURT']=[0,0,0,0,0,0]
confusion_matrix.loc['PETITIONER']=[0,0,0,0,0,0]
confusion_matrix.loc['RESPONDENT']=[0,0,0,0,0,0]
confusion_matrix.loc['LAWYER']=[0,0,0,0,0,0]
confusion_matrix.loc['JUDGE']=[0,0,0,0,0,0]
confusion_matrix.loc['NONE']=[0,0,0,0,0,0]

for i in range(len(testing_data_opt)):
    text=testing_data_opt[i]['text']
    doc=nlp_ner(text)
    #print("###testing###")
    #print(testing_data[i]['entities'])
    for ent in doc.ents:
        flag=0
        #print(ent.start_char,ent.end_char,ent.label_)
        for astart,aend,arole in testing_data_opt[i]['entities']:
            if(astart==ent.start_char and aend==ent.end_char):
                confusion_matrix[ent.label_][arole]+=1
                flag=1
                break
        if(flag==0):
            confusion_matrix[ent.label_]['NONE']+=1
    for astart,aend,arole in testing_data_opt[i]['entities']:
        flag=0
        for ent in doc.ents:
            if(astart==ent.start_char and aend==ent.end_char):
                flag=1
                break
        if(flag==0):
            confusion_matrix['NONE'][arole]+=1
print(confusion_matrix)

precision_dict={}
precision_dict['COURT']=0
precision_dict['PETITIONER']=0
precision_dict['RESPONDENT']=0
precision_dict['LAWYER']=0
precision_dict['JUDGE']=0
precision_dict['NONE']=0

roles=['COURT','PETITIONER','RESPONDENT','LAWYER','JUDGE','NONE']
rolect=0
for role in roles:
    rolesum=0
    print(confusion_matrix[role])
    for i in confusion_matrix[role]:
        rolesum+=i
    precision_dict[role]=(confusion_matrix[role][rolect])/rolesum
    rolect+=1
print(precision_dict)

Token indices sequence length is longer than the specified maximum sequence length for this model (987 > 512). Running this sequence through the model will result in indexing errors


           COURT PETITIONER RESPONDENT LAWYER JUDGE NONE
COURT        253          0          0      0     0   32
PETITIONER     1        526         88      9     0   64
RESPONDENT     0         13        783     18     3  120
LAWYER         0          2          4    896     2   74
JUDGE          1          4          3      5   453   28
NONE          43         30        223     39    15    0
COURT         253
PETITIONER      1
RESPONDENT      0
LAWYER          0
JUDGE           1
NONE           43
Name: COURT, dtype: object
COURT           0
PETITIONER    526
RESPONDENT     13
LAWYER          2
JUDGE           4
NONE           30
Name: PETITIONER, dtype: object
COURT           0
PETITIONER     88
RESPONDENT    783
LAWYER          4
JUDGE           3
NONE          223
Name: RESPONDENT, dtype: object
COURT           0
PETITIONER      9
RESPONDENT     18
LAWYER        896
JUDGE           5
NONE           39
Name: LAWYER, dtype: object
COURT           0
PETITIONER      0
RESPONDENT    

In [ ]:
# doc = nlp_ner()


# spacy.displacy.render(doc, style="ent", jupyter=True)


In [ ]:
doc = nlp_ner('''The Honble Sri Justice U. Durga Prasad Rao           

M.A.C.M.A No.1656 of 2010  

05-11-2014 

Bajaj Allianz General Insurance Co. Ltd, Rep by its Regional Manager, Begumpet,
Hyd..... Appellant

M. Sreedevi and others.. Respondents  

Counsel for Appellant                   : Sri T. Mahender Rao

Counsel for Respondent Nos.1 to 5: Sri Mohd. Yousuf 
 Counsel for Respondent No.6    : Sri M. Achutha Reddy 

<Gist:

>Head Note: 

? Cases referred:
1)      2008 Acj 2855  
2)      2008 Acj 1307  
3)      2003(5) Ald 162 
4)      2004 Acj 1 (Sc) 
5)      1985 Acj 1397 (Sc)  
6)      1995 Acj 358 (Ap)  
7)      2009 Acj 88 (Ap) 


Honble Sri Justice U. Durga Prasad Rao          
M.A.C.M.A.No.1656 of 2010  

Judgment''')


spacy.displacy.render(doc, style="ent", jupyter=True)


/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In [ ]:
testing_data_opt[69]['entities']

[(7, 40, 'COURT'),
 (272, 285, 'PETITIONER'),
 (444, 458, 'RESPONDENT'),
 (577, 665, 'RESPONDENT'),
 (676, 758, 'RESPONDENT'),
 (765, 792, 'RESPONDENT'),
 (880, 891, 'PETITIONER'),
 (1016, 1028, 'PETITIONER'),
 (1266, 1277, 'LAWYER'),
 (1300, 1314, 'LAWYER'),
 (1333, 1344, 'LAWYER'),
 (1386, 1399, 'LAWYER'),
 (1439, 1451, 'LAWYER'),
 (1489, 1502, 'LAWYER'),
 (1546, 1560, 'LAWYER'),
 (1583, 1594, 'LAWYER'),
 (1613, 1623, 'LAWYER'),
 (1638, 1654, 'LAWYER'),
 (1673, 1692, 'LAWYER'),
 (1732, 1745, 'LAWYER'),
 (1847, 1859, 'JUDGE'),
 (1898, 1910, 'JUDGE')]

In [ ]:
doc = nlp_ner('''In The High Court Of Judicature At Patna

                    Letters Patent Appeal No.2294 of 2015
                                 Arising out of
                 Civil Writ Jurisdiction Case No. 17721 of 2015
===========================================================
Amrawati Devi, wife of Sri Dinanath Yadav, Resident of Mohallla- Manik Chand Talab, P.O. Anishabad, P.S. Gardanibagh, District- Patna. 

 .... .... Appellant Versus
1. The State of Bihar through the Principal Secretary, Urban Development and Town Planning Department, Government of Bihar, Patna. 

2. The Principal Secretary, Urban Development and Town Planning Department, Government of Bihar. 

3. The Principal Secretary, Department of Law and Justice Department, Government of Bihar. 

4. Patna Municipal Corporation through its Chief Executive Officer, Maurya Lok Building, Bailey Road, Patna. 

5. Mr. Binod Kumar, son of Sri Janak Prasad Singh, Resident of Mohalla- Mohanpur, Punaichak, P.O and P.S. Shastri Nagar, District- Patna. 

6. Roop Narayan, son of Shri Ram Narayan Prasad, Resident of Mohalla- Mehandiganj, P.O and P.S. Patna City, District-Patna, Bihar. 

 .... .... Respondents =========================================================== Appearance :
For the Appellant : Mr. Vinod Kanth, Sr. Advocate. 

 Mr. S.B.K. Manglam, Advocate. 

 Mr. Ravi Ranjan, Advocate. 

For the State         : Mr. Lalit Kishore, P.A.A.G.
                         Mr. U.S.S. Singh, G.P.-I
                         Mr. R.K. Chandram, A.C. to G.P.-I
For Respondent No. 6 : Mr. Jitendra Singh, Sr. Advocate. 

 Mr. Piyush Lall, Advocate. 

 Mr. Yash Singh, Advocate Mr. Tej Pratap Singh, Advocate. 

 Mr. Kamal Kishore Singh, Advocate. 

For the Corporation : Mr. Prasoon Sinha, Advocate. =========================================================== Coram: Honourable Mr. Justice I. A. Ansari, A.C.J. 

 and Honourable Mr. Justice Hemant Gupta C.A.V. Judgment''')


#spacy.displacy.render(doc, style="ent", jupyter=True)

for ent in doc.ents:
  print(f"Entity {ent} found with start at {ent.start_char} and end at {ent.end_char}")
  


/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


Entity High Court Of Judicature At Patna found with start at 7 and end at 40
Entity Amrawati Devi found with start at 272 and end at 285
Entity Dinanath Yadav found with start at 299 and end at 313
Entity State of Bihar found with start at 444 and end at 458
Entity Principal Secretary, Urban Development and Town Planning Department, Government of Bihar found with start at 577 and end at 665
Entity Principal Secretary, Department of Law and Justice Department, Government of Bihar found with start at 676 and end at 758
Entity Patna Municipal Corporation found with start at 765 and end at 792
Entity Binod Kumar found with start at 880 and end at 891
Entity Roop Narayan found with start at 1016 and end at 1028
Entity Vinod Kanth found with start at 1266 and end at 1277
Entity S.B.K. Manglam found with start at 1300 and end at 1314
Entity Ravi Ranjan found with start at 1333 and end at 1344
Entity Lalit Kishore found with start at 1386 and end at 1399
Entity U.S.S. Singh found with start at

In [ ]:
testing_data_opt[24]['entities']

[(85, 119, 'COURT'),
 (435, 460, 'PETITIONER'),
 (614, 637, 'RESPONDENT'),
 (773, 798, 'PETITIONER'),
 (879, 887, 'PETITIONER'),
 (997, 1035, 'PETITIONER'),
 (1276, 1296, 'RESPONDENT'),
 (1598, 1617, 'LAWYER'),
 (1658, 1668, 'LAWYER'),
 (1841, 1853, 'JUDGE'),
 (1858, 1866, 'JUDGE')]

In [ ]:
doc = nlp_ner('''1                                                       


                   In The High Court Of Judicature At Bombay,
                             Nagpur Bench, Nagpur.




                                                                                                         
                                                                                
                                   Cri. Appeal No.418 Of 2005.


    1. Motiram Marotrao Bhongade




                                                                               
       aged about 70 years, 
       Occupation : Cultivator,

    2. Sanjay Motiram Bhongade,
       aged about 34 years, 




                                                               
       Occupation : Service,

    3. Laxmibai Motiram Bhongade,
       aged about 60 years, 
                                          
    4. Leelabai w/o. Bablu @ Rooprao Falke,
                                         
       aged about 34 years, 

         All residents of Satrapur, P.S. Kanhan,
         District : Nagpur. 
             


                                                                                           .... Appellants.
          



                                                  // Versus //

    The State of Maharashtra,
    Through Police Station, Kanhan, 





    District : Nagpur. 

                                                                                             ....Respondent.
    --------------------------------------------------------------------------------------------------------------
    Shri Shashank V. Manohar, Advocate for Appellants.





    Mrs. K.S. Joshi, A.P.P. for Respondent.
    --------------------------------------------------------------------------------------------------------------

                      Coram  :  A.P. Lavande And P.D.Kode, Jj.
                      Date Of Reserving Judgment''')

spacy.displacy.render(doc, style="ent", jupyter=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (987 > 512). Running this sequence through the model will result in indexing errors
/usr/local/lib/python3.7/dist-packages/torch/amp/autocast_mode.py:198: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')
